In [7]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%sql sqlite:///insurancedb.db


### US HEALTHCARE INSURANCE DATABASE (6 Tables Only)


### 1. Insurance Plans (Bronze, Silver, Gold, etc.)

####  Purpose: Describes insurance products/benefit designs,One row per plan.

In [3]:
%%sql
CREATE TABLE plans (
    plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    plan_name TEXT NOT NULL,
    insurer TEXT NOT NULL,
    plan_type TEXT, 
    metal_level TEXT,
    monthly_premium DECIMAL(8, 2) NOT NULL
);

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table plans already exists
[SQL: CREATE TABLE plans (
    plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    plan_name TEXT NOT NULL,
    insurer TEXT NOT NULL,
    plan_type TEXT, 
    metal_level TEXT,
    monthly_premium DECIMAL(8, 2) NOT NULL
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 2.Members (the insured people)

#### Purpose: Insured people (subscribers and dependencies)

In [4]:
%%sql
CREATE TABLE members(
    member_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    dob DATE,
    gender TEXT CHECK(gender IN ('M','F','O')),
    phone TEXT,
    email TEXT,
    join_date DATE DEFAULT(Date('now'))
);


 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table members already exists
[SQL: CREATE TABLE members(
    member_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    dob DATE,
    gender TEXT CHECK(gender IN ('M','F','O')),
    phone TEXT,
    email TEXT,
    join_date DATE DEFAULT(Date('now'))
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 3.Healthcare Providers

#### Purpose: Rendering providers(Clinics, doctors)

In [5]:
%%sql
CREATE TABLE providers (
    provider_id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_name TEXT NOT NULL,
    npi TEXT UNIQUE,        -- National Provider Identifier
    specialty TEXT,
    city TEXT,
    in_network INTEGER DEFAULT 1      -- 1=YES, 0=NO
);

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table providers already exists
[SQL: CREATE TABLE providers (
    provider_id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_name TEXT NOT NULL,
    npi TEXT UNIQUE,        -- National Provider Identifier
    specialty TEXT,
    city TEXT,
    in_network INTEGER DEFAULT 1      -- 1=YES, 0=NO
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 4.Enrollments

#### Purpose: Maps members to plans and records coverage periods. This allows historical queries who was covered when.

In [6]:
%%sql 
CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    member_id INTEGER NOT NULL,
    plan_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE,
    status TEXT DEFAULT 'Active',
    FOREIGN KEY(member_id) REFERENCES members(member_id),
    FOREIGN KEY(plan_id) REFERENCES plans(plan_id)
);

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table enrollments already exists
[SQL: CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    member_id INTEGER NOT NULL,
    plan_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE,
    status TEXT DEFAULT 'Active',
    FOREIGN KEY(member_id) REFERENCES members(member_id),
    FOREIGN KEY(plan_id) REFERENCES plans(plan_id)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 5. Claims 

#### Purpose: One row per claim(header). You also have claim-lines to hold line details

In [7]:
%%sql
CREATE TABLE claims (
    claim_id INTEGER PRIMARY KEY AUTOINCREMENT,
    member_id INTEGER NOT NULL,
    provider_id INTEGER NOT NULL,
    claim_date DATE DEFAULT(Date('now')),
    service_date DATE NOT NULL,
    submitted_amount DECIMAL(10, 2),
    allowed_amount DECIMAL(10, 2),
    paid_amount DECIMAL(10, 2),
    status TEXT DEFAULT 'Pending',      -- e.g., Pending, Paid, Denied,Partial
    denial_reason TEXT,
    FOREIGN KEY(member_id) REFERENCES members(member_id),
    FOREIGN KEY(provider_id) REFERENCES providers(provider_id)
);


 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table claims already exists
[SQL: CREATE TABLE claims (
    claim_id INTEGER PRIMARY KEY AUTOINCREMENT,
    member_id INTEGER NOT NULL,
    provider_id INTEGER NOT NULL,
    claim_date DATE DEFAULT(Date('now')),
    service_date DATE NOT NULL,
    submitted_amount DECIMAL(10, 2),
    allowed_amount DECIMAL(10, 2),
    paid_amount DECIMAL(10, 2),
    status TEXT DEFAULT 'Pending',      -- e.g., Pending, Paid, Denied,Partial
    denial_reason TEXT,
    FOREIGN KEY(member_id) REFERENCES members(member_id),
    FOREIGN KEY(provider_id) REFERENCES providers(provider_id)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 6. claim_lines

#### Purpose: line level detail- each procedure or service element of a claim

In [8]:

%%sql
CREATE TABLE claims_lines (
    line_id INTEGER PRIMARY KEY AUTOINCREMENT,
    claim_id INTEGER NOT NULL,
    procedure_code TEXT,        -- e.g., 99213, 36415
    diagnosis_code TEXT,        -- e.g., J45.909 (Asthma), E11.9 (Diabetes)
    charge DECIMAL(10, 2),
    paid DECIMAL(10, 2) DEFAULT 0,
    FOREIGN KEY(claim_id) REFERENCES claims(claim_id) ON DELETE CASCADE
)

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) table claims_lines already exists
[SQL: CREATE TABLE claims_lines (
    line_id INTEGER PRIMARY KEY AUTOINCREMENT,
    claim_id INTEGER NOT NULL,
    procedure_code TEXT,        -- e.g., 99213, 36415
    diagnosis_code TEXT,        -- e.g., J45.909 (Asthma), E11.9 (Diabetes)
    charge DECIMAL(10, 2),
    paid DECIMAL(10, 2) DEFAULT 0,
    FOREIGN KEY(claim_id) REFERENCES claims(claim_id) ON DELETE CASCADE
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


* Indexing are used to make queries run fast

In [9]:
%%sql
--  INDEXES 

CREATE INDEX idx_claims_member ON claims(member_id);
CREATE INDEX idx_claims_date ON claims(service_date);
CREATE INDEX idx_claims_status ON claims(status);
CREATE INDEX idx_enrollments_member ON enrollments(member_id);
CREATE INDEX idx_enrollments_dates ON enrollments(start_date, end_date);
CREATE INDEX idx_providers_network ON providers(in_network);
CREATE INDEX idx_claim_lines_claim ON claim_lines(claim_id);

-- Final confirmation
SELECT 'Database with 6 tables created successfully!' AS status;

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) index idx_claims_member already exists
[SQL: --  INDEXES 

CREATE INDEX idx_claims_member ON claims(member_id);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


* Inserting data

In [10]:
%%sql 
1. Insurance Plans
INSERT INTO plans (plan_name, insurer, plan_type, metal_level, monthly_premium) VALUES
('Blue Advantage Silver 70', 'Blue Cross', 'PPO', 'Silver', 485.00),
('UnitedHealthcare Gold 80', 'UnitedHealthcare', 'PPO', 'Gold', 720.00),
('Anthem Bronze 60 HMO', 'Anthem', 'HMO', 'Bronze', 320.00),
('Kaiser Permanente Silver', 'Kaiser', 'HMO', 'Silver', 510.00);

2. Members (patients)
INSERT INTO members (first_name, last_name, dob, gender, phone, email, join_date) VALUES
('Sarah', 'Johnson', '1992-04-12', 'F', '555-1001', 'sarah.j@gmail.com', '2024-01-01'),
('Michael', 'Chen', '1985-09-25', 'M', '555-1002', 'mchen@work.com', '2024-01-15'),
('Emma', 'Rodriguez', '1978-11-30', 'F', '555-1003', 'emma.r@gmail.com', '2024-02-01'),
('James', 'Wilson', '1990-06-18', 'M', '555-1004', 'james.w@outlook.com', '2024-03-01');

3. Providers (doctors & hospitals)
INSERT INTO providers (provider_name, npi, specialty, city, in_network) VALUES
('Dr. Lisa Thompson', '1234567890', 'Family Medicine', 'Chicago', 1),
('Chicago General Hospital', '9876543210', 'Emergency Room', 'Chicago', 1),
('Dr. Raj Patel', '1122334455', 'Cardiology', 'Chicago', 1),
('Northside Clinic', '5566778899', 'Pediatrics', 'Evanston', 0),  -- out-of-network
('Dr. Amy Kim', '9988776655', 'OB/GYN', 'Chicago', 1);

4. Enrollments (who is covered by which plan)
INSERT INTO enrollments (member_id, plan_id, start_date, end_date, status) VALUES
(1, 1, '2024-01-01', NULL, 'Active'),        -- Sarah → Blue Cross Silver
(2, 2, '2024-01-15', NULL, 'Active'),        -- Michael → United Gold
(3, 1, '2024-02-01', NULL, 'Active'),        -- Emma → Blue Cross Silver
(4, 3, '2024-03-01', '2024-08-31', 'Terminated'); -- James → Anthem Bronze (left)

5. Claims (real medical bills)
INSERT INTO claims (member_id, provider_id, claim_date, service_date, submitted_amount, allowed_amount, paid_amount, status, denial_reason) VALUES
(1, 1, '2024-03-10', '2024-03-05', 250.00, 180.00, 180.00, 'Paid', NULL),
(1, 2, '2024-04-20', '2024-04-18', 3200.00, 2800.00, 2800.00, 'Paid', NULL),
(2, 3, '2024-05-15', '2024-05-10', 1200.00, 950.00, 950.00, 'Paid', NULL),
(3, 4, '2024-06-01', '2024-05-28', 800.00, 0.00, 0.00, 'Denied', 'Out of Network'),
(1, 1, '2024-07-12', '2024-07-10', 150.00, 0.00, 0.00, 'Denied', 'Prior Authorization Required'),
(2, 1, '2024-08-05', '2024-08-02', 200.00, 160.00, 160.00, 'Paid', NULL);

6. Claim Lines (detailed services on each claim)
INSERT INTO claim_lines (claim_id, procedure_code, diagnosis_code, charge, paid) VALUES
(1, '99213', 'J45.909', 250.00, 180.00),   -- Office visit, Asthma
(2, '99291', 'I21.9', 3200.00, 2800.00),   -- ER critical care, Heart attack
(3, '92920', 'I25.10', 1200.00, 950.00),   -- Angioplasty
(4, '99395', 'Z00.00', 800.00, 0.00),      -- Annual physical (denied - out of network)
(5, '99214', 'E11.9', 150.00, 0.00),       -- Office visit, Diabetes (denied - no prior auth)
(6, '99213', 'R10.9', 200.00, 160.00);     -- Abdominal pain visit

Confirmation
SELECT 'Sample data inserted! 4 members, 6 claims, real denials included' AS status;

 * sqlite:///insurancedb.db
(sqlite3.OperationalError) near "1.": syntax error
[SQL: 1. Insurance Plans
INSERT INTO plans (plan_name, insurer, plan_type, metal_level, monthly_premium) VALUES
('Blue Advantage Silver 70', 'Blue Cross', 'PPO', 'Silver', 485.00),
('UnitedHealthcare Gold 80', 'UnitedHealthcare', 'PPO', 'Gold', 720.00),
('Anthem Bronze 60 HMO', 'Anthem', 'HMO', 'Bronze', 320.00),
('Kaiser Permanente Silver', 'Kaiser', 'HMO', 'Silver', 510.00);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


*  See all plans

In [ ]:
%%sql 
SELECT * FROM plans;

 * sqlite:///insurancedb.db
Done.


plan_id,plan_name,insurer,plan_type,metal_level,monthly_premium
1,Blue Advantage Silver 70,Blue Cross,PPO,Silver,485
2,UnitedHealthcare Gold 80,UnitedHealthcare,PPO,Gold,720
3,Anthem Bronze 60 HMO,Anthem,HMO,Bronze,320
4,Kaiser Permanente Silver,Kaiser,HMO,Silver,510


* See all members

In [ ]:
%%sql
SELECT member_id, first_name, last_name, email FROM members;

 * sqlite:///insurancedb.db
Done.


member_id,first_name,last_name,email
1,Sarah,Johnson,sarah.j@gmail.com
2,Michael,Chen,mchen@work.com
3,Emma,Rodriguez,emma.r@gmail.com
4,James,Wilson,james.w@outlook.com


* See denied claims

In [ ]:
%%sql
SELECT claim_id, member_id, submitted_amount, denial_reason 
FROM claims 
WHERE status = 'Denied';

 * sqlite:///insurancedb.db
Done.


claim_id,member_id,submitted_amount,denial_reason
4,3,800,Out of Network
5,1,150,Prior Authorization Required


* Show all active insurance plans(Shows what plans members can choose from)

In [ ]:
%%sql
SELECT plan_id, plan_name, insurer, monthly_premium 
FROM plans;

 * sqlite:///insurancedb.db
Done.


plan_id,plan_name,insurer,monthly_premium
1,Blue Advantage Silver 70,Blue Cross,485
2,UnitedHealthcare Gold 80,UnitedHealthcare,720
3,Anthem Bronze 60 HMO,Anthem,320
4,Kaiser Permanente Silver,Kaiser,510


* Find all members named "Sarah" (Basic member lookup)

In [15]:
%%sql
SELECT member_id, first_name, last_name, email 
FROM members 
WHERE first_name = 'Sarah';

 * sqlite:///insurancedb.db
Done.


member_id,first_name,last_name,email
1,Sarah,Johnson,sarah.j@gmail.com


* Show all in network doctors in Chicago(Helps members avoid out of network denials)

In [16]:
%%sql
SELECT provider_name, specialty 
FROM providers 
WHERE city = 'Chicago' AND in_network = 1;

 * sqlite:///insurancedb.db
Done.


provider_name,specialty
Dr. Lisa Thompson,Family Medicine
Chicago General Hospital,Emergency Room
Dr. Raj Patel,Cardiology
Dr. Amy Kim,OB/GYN


* Member's Active Plan(Most important for eligibility check)

In [17]:
%%sql
SELECT 
    m.member_id,
    m.first_name || ' ' || m.last_name AS member_name,
    p.plan_name,
    p.insurer,
    e.start_date,
    e.status
FROM members m
JOIN enrollments e ON m.member_id = e.member_id
JOIN plans p ON e.plan_id = p.plan_id
WHERE e.status = 'Active';

 * sqlite:///insurancedb.db
Done.


member_id,member_name,plan_name,insurer,start_date,status
1,Sarah Johnson,Blue Advantage Silver 70,Blue Cross,2024-01-01,Active
2,Michael Chen,UnitedHealthcare Gold 80,UnitedHealthcare,2024-01-15,Active
3,Emma Rodriguez,Blue Advantage Silver 70,Blue Cross,2024-02-01,Active


* Sarah Johnson's Full Claim History(Member Portal View)

Sarah sees her ER visit (paid),office visit(paid),and one denied claim

In [18]:
%%sql
SELECT 
    c.claim_id,
    c.service_date,
    pr.provider_name,
    c.submitted_amount,
    c.paid_amount,
    c.status,
    c.denial_reason
FROM claims c
JOIN members m ON c.member_id = m.member_id
JOIN providers pr ON c.provider_id = pr.provider_id
WHERE m.first_name = 'Sarah' AND m.last_name = 'Johnson'
ORDER BY c.service_date DESC;

 * sqlite:///insurancedb.db
Done.


claim_id,service_date,provider_name,submitted_amount,paid_amount,status,denial_reason
5,2024-07-10,Dr. Lisa Thompson,150,0,Denied,Prior Authorization Required
2,2024-04-18,Chicago General Hospital,3200,2800,Paid,None
1,2024-03-05,Dr. Lisa Thompson,250,180,Paid,None


* All Denied claims with Member & Provider Details(Appeals Team View)

In [19]:
%%sql
SELECT 
    c.claim_id,
    m.first_name || ' ' || m.last_name AS member,
    pr.provider_name,
    c.service_date,
    c.submitted_amount,
    c.denial_reason
FROM claims c
JOIN members m ON c.member_id = m.member_id
JOIN providers pr ON c.provider_id = pr.provider_id
WHERE c.status = 'Denied';

 * sqlite:///insurancedb.db
Done.


claim_id,member,provider_name,service_date,submitted_amount,denial_reason
4,Emma Rodriguez,Northside Clinic,2024-05-28,800,Out of Network
5,Sarah Johnson,Dr. Lisa Thompson,2024-07-10,150,Prior Authorization Required


* Total Amount Paid per Member(High Cost Member Report)

In [20]:
%%sql
SELECT 
    m.first_name || ' ' || m.last_name AS member,
    COUNT(c.claim_id) AS claims_count,
    SUM(c.paid_amount) AS total_paid
FROM members m
LEFT JOIN claims c ON m.member_id = c.member_id AND c.status = 'Paid'
GROUP BY m.member_id, m.first_name, m.last_name
ORDER BY total_paid DESC;

 * sqlite:///insurancedb.db
Done.


member,claims_count,total_paid
Sarah Johnson,2,2980
Michael Chen,2,1110
Emma Rodriguez,0,None
James Wilson,0,None


* In Network vs Out of Network Usage(Network Leakage Report)

In [21]:
%%sql
SELECT
    pr.in_network,
    COUNT(c.claim_id) AS claims,
    SUM(c.submitted_amount) AS total_billed
FROM claims c
JOIN providers pr ON c.provider_id = pr.provider_id
GROUP BY pr.in_network;

 * sqlite:///insurancedb.db
Done.


in_network,claims,total_billed
0,1,800
1,5,5000


In [22]:
%%sql
SELECT
    COUNT(c.claim_id) AS claims,
    SUM(c.submitted_amount) AS total_billed
FROM claims c
JOIN providers pr ON c.provider_id = pr.provider_id
GROUP BY pr.in_network;

 * sqlite:///insurancedb.db
Done.


claims,total_billed
1,800
5,5000


In [3]:
from sqlalchemy import create_engine, text  # import text

# Replace with your PostgreSQL info
user = "postgres"
password = "pass123"
host = "localhost"
port = "5432"
database = "insurance"

pg_engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Test connection
with pg_engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))  # wrap SQL in text()
    print(result.fetchone())

('PostgreSQL 18.1 on x86_64-windows, compiled by msvc-19.44.35219, 64-bit',)


In [4]:
from sqlalchemy import create_engine, inspect
import pandas as pd

# SQLite connection
sqlite_engine = create_engine('sqlite:///insurance.db')

# Use inspector to get table names
inspector = inspect(sqlite_engine)
table_names = inspector.get_table_names()
print("Tables in SQLite:", table_names)

Tables in SQLite: []


In [5]:
sqlite_tables = {table: pd.read_sql_table(table, sqlite_engine) for table in table_names}

# Optional: check first few rows
for name, df in sqlite_tables.items():
    print(f"\nTable: {name}")
    print(df.head())

In [9]:
# Step 0: Imports
from sqlalchemy import create_engine, inspect, text
import pandas as pd

# Step 1: Define connections

# SQLite source
sqlite_engine = create_engine('sqlite:///insurancedb.db')

# PostgreSQL target — replace with your info
pg_user = "postgres"
pg_password = "pass123"
pg_host = "localhost"
pg_port = "5432"
pg_database = "insurance"

pg_engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}')

# Step 2: Get all SQLite tables
inspector = inspect(sqlite_engine)
table_names = inspector.get_table_names()
print("Tables in SQLite:", table_names)

# Step 3: Read tables into pandas and migrate to PostgreSQL
for table in table_names:
    # Read table from SQLite
    df = pd.read_sql_table(table, sqlite_engine)
    
    # Push to PostgreSQL
    df.to_sql(table, pg_engine, if_exists='replace', index=False)
    
    print(f"Migrated table: {table}")

# Step 4: Test PostgreSQL connection (optional)
with pg_engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print("\nPostgreSQL version:", result.fetchone())

Tables in SQLite: ['claims', 'claims_lines', 'enrollments', 'members', 'plans', 'providers']
Migrated table: claims
Migrated table: claims_lines
Migrated table: enrollments
Migrated table: members
Migrated table: plans
Migrated table: providers

PostgreSQL version: ('PostgreSQL 18.1 on x86_64-windows, compiled by msvc-19.44.35219, 64-bit',)
